# Computer Vision with the CIFAR-10 Dataset

In [20]:
#for Reproducibility Purposes we set a seed
import tensorflow as tf
from numpy.random import seed
seed(1)
tf.compat.v1.set_random_seed(1)

#Importing the CIFAR10 Dataset
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#tf.keras.utils.to_categorical

In [2]:
#Batch_size for Conveniece:
batch_size = 50

In [3]:
#Importing the dataset,https://www.cs.toronto.edu/~kriz/cifar.html
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

#Transforming the dataset into float32 arrays
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
#Here I normalize the pixels by deviding it into the maximum value which is 255
X_train /= 255
X_test /= 255

In [7]:
#They are RGB images in 3 channels!
print('CIFAR-10 Dataset Shape:')
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

CIFAR-10 Dataset Shape:
X_train: (50000, 32, 32, 3)
Y_train: (50000, 10)
X_test:  (10000, 32, 32, 3)
Y_test:  (10000, 10)


In [4]:
#This are the lalbels 0,10 . The number 4 will be depicted as 4 is mapped to [0, 0, 0,0, 1, 0, 0, 0, 0, 0]
classes = 10
Y_train = tf.keras.utils.to_categorical(Y_train, classes)
Y_test = tf.keras.utils.to_categorical(Y_test, classes)


In [5]:
#We implement an image generator that allows us to augment the dataset.We are rotating the images 90 degrees to creat extra data.
data_generator = ImageDataGenerator(rotation_range=90,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    featurewise_center=True,
#We basically just x=x-miu/standrd deviation. It it useful to keep values of the matrix close by and achieve vetter predictions when sampling. Is like removing outliers                              
                                    featurewise_std_normalization=True,
                                    horizontal_flip=True)
#We just run the algorithm over the whole dataset
data_generator.fit(X_train)

In [6]:
# standardize the test set
for i in range(len(X_test)):
    X_test[i] = data_generator.standardize(X_test[i])


In [22]:
# defining the network

model=Sequential()
#1. First Convolutional Layer. I padding=1 the borders of all images so they match.Input_shape=X_train.shape[1:]=(32,32,3)
model.add(Convolution2D(filters=32,
                     kernel_size=(3,3),
                     padding='same',
                     input_shape=(32,32,3)))
#2. Now I apply Batchnormalization function: It normalizes the outputs of the hidden layer for each
#mini-batch (hence the name) in a way, which maintains its mean activation value close to 0,
#and its standard deviation close to 1. Si it trains faster and has larger learning rates
model.add(BatchNormalization())
#3. We add an activation layer, in this case exponential linear unit (ELU), why?
model.add(Activation('elu'))
#4. We add a second activation layer no need to define the shape here
model.add(Convolution2D(filters=32,
                     kernel_size=(3,3),
                     padding='same'))
#5. Add more normalizationhttps://towardsdatascience.com/batch-normalization-theory-and-how-to-use-it-with-tensorflow-1892ca0173ad
model.add(BatchNormalization())
#6. Another Activation Function
model.add(Activation('elu'))
#7. Do pooling to reduce dimensionality
model.add(MaxPooling2D(pool_size=(2,2)))
#8. We randomly drop a neuron while training the model with 20% probability to avoid overfitting.
model.add(Dropout(0.2))
#9. We apply a third Convolution Layer and fourth
model.add(Convolution2D(filters=64,
                     kernel_size=(3,3),
                     padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Convolution2D(filters=64,
                     kernel_size=(3,3),
                     padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
#10. Sigo Poninedo layers
model.add(Convolution2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(Convolution2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

#11. I flatten the input, create a normal neural network with 10 hidden layers, and an activatoion function
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [24]:
# Now we decide the loss function and optimization method

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


In [25]:
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_17 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_18 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)       

In [26]:
model.fit_generator(generator=data_generator.flow(x=X_train,y=Y_train,batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=3,
                    validation_data=(X_test, Y_test),
                    workers=4)

/home/rviccinig/anaconda3/envs/test-ai/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
1000/1000 [==============================] - 128s 127ms/step - loss: 2.2078 - accuracy: 0.2794 - val_loss: 1.6099 - val_accuracy: 0.4390
Epoch 2/3
1000/1000 [==============================] - 127s 127ms/step - loss: 1.5986 - accuracy: 0.4239 - val_loss: 1.4917 - val_accuracy: 0.4969
Epoch 3/3
1000/1000 [==============================] - 128s 128ms/step - loss: 1.4185 - accuracy: 0.4907 - val_loss: 1.4322 - val_accuracy: 0.5131
